In [0]:
pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
data = sc.textFile("/FileStore/tables/spam_or_not_spam-1.csv")

In [0]:
#train test split
test, train = data.randomSplit(weights=[0.3, 0.7], seed=1)

In [0]:
split_classifier=train.map(lambda x: x.split(","))

In [0]:
split_classifier.collect()

Out[5]: [[' date wed NUMBER aug NUMBER NUMBER NUMBER NUMBER NUMBER from chris garrigues cwg dated NUMBER NUMBERfaNUMBERd deepeddy com message id NUMBER NUMBER tmda deepeddy vircio com i can t reproduce this error for me it is very repeatable like every time without fail this is the debug log of the pick happening NUMBER NUMBER NUMBER pick_it exec pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER NUMBER NUMBER exec pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER NUMBER NUMBER ftoc_pickmsgs NUMBER hit NUMBER NUMBER NUMBER marking NUMBER hits NUMBER NUMBER NUMBER tkerror syntax error in expression int note if i run the pick command by hand delta pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER hit that s where the NUMBER hit comes from obviously the version of nmh i m using is delta pick version pick nmh NUMBER NUMBER NUMBER compiled on URL at sun mar NUMB

In [0]:
#split between normal mail and spam 
classifier_normal=split_classifier.filter(lambda x:x[1]=='0')

In [0]:
length_normal=len(classifier_normal.collect())

In [0]:
length_normal

Out[8]: 1739

In [0]:
classifier_spam=split_classifier.filter(lambda x:x[1]=='1')

In [0]:
length_spam=len(classifier_spam.collect())

In [0]:
length_spam

Out[11]: 362

In [0]:
#split each word
classifier_normal_flatmap=classifier_normal.flatMap(lambda x:x[0].split(" "))

In [0]:
filtered_rdd_normal = classifier_normal_flatmap.filter(lambda word: word.lower() not in stop_words)

In [0]:
classifier_spam_flatmap=classifier_spam.flatMap(lambda x:x[0].split(" "))

In [0]:
filtered_rdd_spam = classifier_spam_flatmap.filter(lambda word: word.lower() not in stop_words)

In [0]:
#add one to each word
classifier_normal_flatmap_one=filtered_rdd_normal.map(lambda x: (x,1))

In [0]:
classifier_spam_flatmap_one=filtered_rdd_spam.map(lambda x: (x,1))

In [0]:
#number of normal words
length_normal_word=len(classifier_normal_flatmap_one.collect())

In [0]:
#number of spam words
length_spam_words=len(classifier_spam_flatmap_one.collect())

In [0]:
#adding all the common words
classifier_normal_add=classifier_normal_flatmap_one.reduceByKey(lambda x,y:x+y)

In [0]:
classifier_spam_add=classifier_spam_flatmap_one.reduceByKey(lambda x,y:x+y)

In [0]:
#finding the probability by dividing 
classifier_normal_add_2values=classifier_normal_add.map(lambda x:(x[0],x[1]/length_normal_word))

In [0]:
classifier_spam_add_2values=classifier_spam_add.map(lambda x:(x[0],x[1]/length_spam_words))

In [0]:
words_rdd = test.map(lambda x:x)

In [0]:
#length of the test dataset
test_len=len(words_rdd.collect())

In [0]:
#split based on ,
split_classifier_test=words_rdd.map(lambda x: x.split(","))

In [0]:
#adding index for each test case
testing_data_with_key=split_classifier_test.zipWithIndex()

In [0]:
#remove the classification and just keep the sentence and index
testing_data_transform=testing_data_with_key.map(lambda x:(x[1],x[0][0]))

In [0]:
#keeping the index and classification 
final_value_to_test=testing_data_with_key.map(lambda x:(x[1],x[0][1]))

In [0]:
#split each word
testing_data_transform_oneadded=testing_data_transform.flatMap(lambda x:((x[0],(i)) for i in x[1].split(" ")))

In [0]:
test_filtered_rdd = testing_data_transform_oneadded.filter(lambda word: word[1].lower() not in stop_words)

In [0]:
#index with word order changed for joining rdd
new_transfomred_before_reduce=test_filtered_rdd.map(lambda x:(x[1],x[0]))

In [0]:
#joined normal values
joined_value_normal=classifier_normal_add_2values.rightOuterJoin(new_transfomred_before_reduce)

In [0]:
joined_value_spam=classifier_spam_add_2values.rightOuterJoin(new_transfomred_before_reduce)

In [0]:
joined_value_spam_zero_transform=joined_value_spam.map(lambda x: (x[0],(1 if x[1][0] is None else x[1][0],x[1][1])))

In [0]:
joined_value_normal_zero_transform=joined_value_normal.map(lambda x: (x[0],(1 if x[1][0] is None else x[1][0],x[1][1])))

In [0]:
#taking the id and probability value after mapping
prob_key=joined_value_normal_zero_transform.map(lambda x:(x[1][1],x[1][0]))

In [0]:
prob_key_spam=joined_value_spam_zero_transform.map(lambda x:(x[1][1],x[1][0]))

In [0]:
#All small values are multiplied which results in zero. So we are transforming the data
import math
log_transform_normal=prob_key.map(lambda x:(x[0],math.log(x[1])))

In [0]:
#as we have changed it to log we are adding the value instead of multiplying 
prob_correct_class=log_transform_normal.reduceByKey(lambda x, y: x + y)

In [0]:
log_transform_spam=prob_key_spam.map(lambda x:(x[0],math.log(x[1])))

In [0]:
prob_spam_class=log_transform_spam.reduceByKey(lambda x, y: x + y)

In [0]:
#adding the prior probabilty with the existing probability 
final_spam_ouput=prob_correct_class.map(lambda x:(x[0],x[1]+(math.log(length_spam/(length_spam+length_normal)))))

In [0]:
final_correct_ouput=prob_spam_class.map(lambda x:(x[0],x[1]+(math.log(length_normal/(length_spam+length_normal)))))

In [0]:
#joining the final correct and spam probability
out_after_all_join=final_spam_ouput.fullOuterJoin(final_correct_ouput)

In [0]:
out_after_all_join_zero_added=out_after_all_join.map(lambda x: (x[0],(0 if x[1][0] is None else x[1][0],(0 if x[1][1] is None else x[1][1]))))

In [0]:
#True valu will be displayed if prob(spam)>prob(correct)
final_op_true_or_false=out_after_all_join_zero_added.map(lambda x:(x[0],x[1][0]>x[1][1]))

In [0]:
#joiing the labels in the test dataset with the values we obtained
joined_op=final_op_true_or_false.join(final_value_to_test)

In [0]:
#if the label is turn it to true else mark it as false
final_op=joined_op.map(lambda x:(x[0],(x[1][0], True if x[1][1] == "1" else False)))

In [0]:
#checking if x[1][0]==x[1][1]
final_op_for_acc=final_op.map(lambda x:x[1][0]==x[1][1])

In [0]:
#dividing the correct classification by total length of test dataset
accurcay=len(final_op_for_acc.filter(lambda x:x==True).collect())/test_len

In [0]:
accurcay*100

Out[52]: 84.38538205980066

In [0]:
prior_prob_spam=(length_spam)/(length_spam+length_normal)

In [0]:
prior_prob_spam

Out[54]: 0.17229890528319847

In [0]:
prior_prob_correct=(length_normal)/(length_spam+length_normal)

In [0]:
prior_prob_correct

Out[56]: 0.8277010947168015